In [ ]:
import os
import torch
from torch import nn, optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load the full dataset
data_dir = 'chessDataset'
full_dataset = datasets.ImageFolder(data_dir, transform=transform)

# Define class names based on folder structure
class_names = [
    'black_bishop', 'black_king', 'black_knight', 'black_pawn', 'black_queen', 'black_rook',  # Black pieces
    'empty',  # Empty squares
    'white_bishop', 'white_king', 'white_knight', 'white_pawn', 'white_queen', 'white_rook'  # White pieces
]

# Split dataset into training and validation
train_size = int(0.8 * len(full_dataset))  # 80% for training
valid_size = len(full_dataset) - train_size  # 20% for validation
train_data, valid_data = random_split(full_dataset, [train_size, valid_size])

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=32, shuffle=True)

# Define model (using a pre-trained model like ResNet for feature extraction)
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, len(class_names))  # Adjust output layer to number of classes
model = model.to(device)

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

# Training loop
def train_model(model, train_loader, valid_loader, criterion, optimizer, epochs=10):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        # Validation
        model.eval()
        valid_loss = 0.0
        accuracy = 0
        with torch.no_grad():
            for inputs, labels in valid_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                valid_loss += loss.item()
                _, preds = torch.max(outputs, 1)
                accuracy += (preds == labels).sum().item()

        # Print epoch results
        print(f"Epoch {epoch+1}/{epochs}")
        print(f"Train Loss: {running_loss/len(train_loader):.4f}")
        print(f"Validation Loss: {valid_loss/len(valid_loader):.4f}")
        print(f"Validation Accuracy: {accuracy / len(valid_data):.4f}")

# Train the model
train_model(model, train_loader, valid_loader, criterion, optimizer, epochs=10)

# Save the model
torch.save(model.state_dict(), 'chess_piece_classifier.pth')


C:\Users\Nishant Mohan\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Nishant Mohan\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/10
Train Loss: 0.8590
Validation Loss: 0.2479
Validation Accuracy: 0.9500
Epoch 2/10
Train Loss: 0.1734
Validation Loss: 0.0530
Validation Accuracy: 1.0000
Epoch 3/10
Train Loss: 0.0678
Validation Loss: 0.0238
Validation Accuracy: 1.0000
Epoch 4/10
Train Loss: 0.0441
Validation Loss: 0.0157
Validation Accuracy: 1.0000
Epoch 5/10
Train Loss: 0.0298
Validation Loss: 0.0118
Validation Accuracy: 1.0000
